# Baselines

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import helper
import matplotlib
import matplotlib.pyplot as plt
import w2v_d2v
import dataset_helper
import preprocessing
import classifier_baseline

In [3]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'limit_output': 1000});

In [4]:
datasets = dataset_helper.get_all_datasets(preprocessed = True)

In [58]:
clfs = classifier_baseline.get_classifiers(iterations=5)
#clfs = {clf_name: clf for clf_name, clf in clfs.items() if clf_name == 'Doc2VecClassifier'}
for dataset_name, (X, Y) in datasets.items():
    if not dataset_name.endswith('-ana'): continue
    #if not dataset_name.startswith('web'): continue
    print('{} {:<14} #docs {}'.format('=' * 30, dataset_name, len(X)))
    data_train_X, data_test_X, data_train_Y, data_test_Y = dataset_helper.split_dataset(X, Y)
    vectors_trans_train, vectors_trans_test = classifier_baseline.vectorize_text(data_train_X, data_test_X)
    pred_results = classifier_baseline.get_predictions_for_classifiers(vectors_trans_train, vectors_trans_test, data_train_Y, data_test_Y, data_train_X, data_test_X, clfs = clfs)
    results = classifier_baseline.get_f1_score_for_clfs(pred_results, data_train_Y, data_test_Y)
    display(pd.DataFrame(results).T.test.to_frame().sort_values('test'))
    #print('{}>    Best for dataset: {} is classifier: {} with F1-score of {}'.format('=' * 10, dataset_name, best_clf, best_clf_score['test']))

============================== webkb-ana      #docs 4168
Currently training classifier: LogisticRegression             (5/5)


/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,test
MostFrequentLabel,0.140207
LogisticRegression,0.847752
Perceptron,0.851879
SGDClassifier,0.858206
PassiveAggressiveClassifier,0.862844


============================== mini20-ana     #docs 1999
Currently training classifier: LogisticRegression             (5/5)


,test
MostFrequentLabel,0.004762
Perceptron,0.680719
SGDClassifier,0.753403
LogisticRegression,0.753542
PassiveAggressiveClassifier,0.772648


...

,test
MostFrequentLabel,0.028558
LogisticRegression,0.408340
Perceptron,0.468812
PassiveAggressiveClassifier,0.535995
SGDClassifier,0.562766


============================== r52-ana        #docs 9100
Currently training classifier: LogisticRegression             (5/5)


,test
MostFrequentLabel,0.011580
LogisticRegression,0.263905
Perceptron,0.731829
SGDClassifier,0.774228
PassiveAggressiveClassifier,0.781498


============================== r8-ana         #docs 7674
Currently training classifier: LogisticRegression             (5/5)


,test
MostFrequentLabel,0.084591
LogisticRegression,0.757962
SGDClassifier,0.912691
PassiveAggressiveClassifier,0.921493
Perceptron,0.923829


## Train Doc2Vec and Word2Vec classifier

### Pre-process docs for d2v and w2v

In [59]:
w2v_data_train = [w2v_d2v.w2v_preproess(doc) for doc in data_train_X]
w2v_data_test = [w2v_d2v.w2v_preproess(doc) for doc in data_test_X]

### Train Doc2Vec

In [127]:
model_d2v = w2v_d2v.train_d2v(w2v_data_train, data_train_Y, iterations = 5)

In [128]:
scores = w2v_d2v.score_d2v(clfs, data_train_Y, data_test_Y, model_d2v, w2v_data_train, w2v_data_test, steps = 1)

Inferred train vectors
Inferred test vectors


/Users/davidgengenbach/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [129]:
scores

{'LogisticRegression': {'test': 0.43375480433862312,
  'train': 0.46532463660461809},
 'MostFrequentLabel': {'test': 0.0046355498721227621,
  'train': 0.0045970578829549087},
 'PassiveAggressiveClassifier': {'test': 0.26940955676406886,
  'train': 0.30912823359685837},
 'Perceptron': {'test': 0.006102700076132176, 'train': 0.010673521739491008},
 'SGDClassifier': {'test': 0.35247761491407559, 'train': 0.38979592547038749}}